In [30]:
!pip3 install git+https://github.com/mr7495/RetinaNet
!pip3 install keras.applications
!pip3 install --upgrade tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/mr7495/RetinaNet to /tmp/pip-req-build-289u_6vl
  Running command git clone --filter=blob:none --quiet https://github.com/mr7495/RetinaNet /tmp/pip-req-build-289u_6vl
  Resolved https://github.com/mr7495/RetinaNet to commit 6a85690f62f9e504db99dcd04391fa790690c498
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tensorflow-io-gcs-filesystem>=0.21.0 (from tensorflow) (from versions: none)
ERROR: No matching distribution found for tensorflow-io-gcs-filesystem>=0.21.0


In [31]:
import keras
import numpy as np
import cv2
import os
import random
import shutil
import pandas as pd
import csv
import zipfile
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import RandomNormal
import keras.backend as k
from sklearn.utils import shuffle
import io
from PIL import Image as pil_image
from keras_retinanet import layers
import keras.backend as k
import keras_retinanet

ModuleNotFoundError: No module named 'keras.applications.resnet50'

In [26]:
 shape=(512,512,1) 

In [27]:
!mkdir 'selected_data'
data_path='SR_2'
data_files=[]
for r,d,f in os.walk(data_path):
  for file in f:
    if '.tif' in file:
      data_files.append(os.path.join(r,file)) 

selected=[]          
zero=[]
names=[]
for img_path in data_files:
    names.append(img_path)
    pixel=cv2.imread(img_path,cv2.IMREAD_UNCHANGED )
    sp=pixel[240:340,120:370] 
    counted_zero=0
    for i in np.reshape(sp,(sp.shape[0]*sp.shape[1],1)):
        if i<300:
            counted_zero+=1
    zero.append(counted_zero)
min_zero=min(zero)
max_zero=max(zero)
threshold=(max_zero-min_zero)/1.5
indices=np.where(np.array(zero)>threshold) 
selected_names=np.array(names)[indices]


for selected_img in selected_names:
    shutil.copy(selected_img,'selected_data')

mkdir: cannot create directory ‘selected_data’: File exists


ValueError: min() arg is an empty sequence

In [28]:
Path= 'FPN-fold1.hdf5'
selected_imgs_path=[]
for r,d,f in os.walk('selected_data'):
  for file in f:
    if '.tif' in file:
      selected_imgs_path.append(os.path.join(r,file)) 

k.clear_session()
custom_object={'UpsampleLike': keras_retinanet.layers._misc.UpsampleLike} 
net=keras.models.load_model(Path, custom_objects=custom_object)


covid_label=0 
normal_label=1  
normal_predicted=0
covid_predicted=0
for img_name in selected_imgs_path: 
  img=cv2.imread(img_name,cv2.IMREAD_UNCHANGED)
  pred_ind=np.argmax(net.predict(np.expand_dims(np.expand_dims(img,axis=0),axis=3))[0])
  if pred_ind==normal_label:
    normal_predicted+=1
  else:
    covid_predicted+=1

all_imgs_num=len(selected_imgs_path) 
if covid_predicted>=0.1*all_imgs_num: 
  print('The patient is identified as infected to COVID-19')
else:
  print('The patient is identified as normal')

The patient is identified as infected to COVID-19
